# Table Extractor Multimodal
The purpose of this program is extract the tables contained in a image or pdf.

The pipeline is the follows:
- First the program convert the pdf to image
- The image is analized by the llm multimodal
- It is extracted all the tables in format csv
- It is preprocessed the output of the llm into csv and saved all individual tables.


In [ ]:
#Package installation
!pip install git+https://github.com/huggingface/transformers.git
!pip install torch, accelerate, bitsandbyte, sentencepiece, pillow
!pip install spaces


In [ ]:
import gradio as gr
import os
import torch
from transformers import AutoProcessor, MllamaForConditionalGeneration, TextStreamer
from PIL import Image

# Check if we're running in a Hugging Face Space and if SPACES_ZERO_GPU is enabled
IS_SPACES_ZERO = os.environ.get("SPACES_ZERO_GPU", "0") == "1"
IS_SPACE = os.environ.get("SPACE_ID", None) is not None
IS_GDRVIE = True

# Determine the device (GPU if available, else CPU)
device = "cuda" if torch.cuda.is_available() else "cpu"
LOW_MEMORY = os.getenv("LOW_MEMORY", "0") == "1"
print(f"Using device: {device}")
print(f"Low memory mode: {LOW_MEMORY}")

# Get Hugging Face token from environment variables
HF_TOKEN = os.environ.get('HF_TOKEN')

# Define the model name
model_name = "Llama-3.2-11B-Vision-Instruct"
if IS_GDRVIE:
    # Define the path to the model directory in your Google Drive
    model_path = "/content/drive/MyDrive/models/" + model_name
    model = MllamaForConditionalGeneration.from_pretrained(
        model_path,
        torch_dtype=torch.bfloat16,
        device_map="auto",
    )
    processor = AutoProcessor.from_pretrained(model_path)
else:
    model_name = "ruslanmv/" + model_name
    model = MllamaForConditionalGeneration.from_pretrained(
        model_name,
        use_auth_token=HF_TOKEN,
        torch_dtype=torch.bfloat16,
        device_map="auto",
    )
    processor = AutoProcessor.from_pretrained(model_name, use_auth_token=HF_TOKEN)

# Tie the model weights to ensure the model is properly loaded
if hasattr(model, "tie_weights"):
    model.tie_weights()

# Stream LLM response generator
def stream_response(inputs):
    streamer = TextStreamer(tokenizer=processor.tokenizer)
    for token in model.generate(**inputs, max_new_tokens=2000, do_sample=True, streamer=streamer):
        yield processor.decode(token, skip_special_tokens=True)

def predict(message, image):
    # Prepare the input messages
    messages = [
        {"role": "user", "content": [
            {"type": "image"},  # Specify that an image is provided
            {"type": "text", "text": message}  # Add the user-provided text input
        ]}
    ]

    # Create the input text using the processor's chat template
    input_text = processor.apply_chat_template(messages, add_generation_prompt=True)

    # Process the inputs and move to the appropriate device
    inputs = processor(image, input_text, return_tensors="pt").to(device)

    # Return a streaming generator of responses
    return stream_response(inputs)

In [ ]:
# prompt: Can you create a python code that takes the page_1.png file and pass the message "Please extract the first table in csv format of the image" and also display the results in streaming, such we can see how is generated
from PIL import Image
# Assuming 'page_1.png' is in the current directory or you provide the correct path
image = Image.open("page_1.png")
example='''Table 1:
value1,value2,value3

Table 2:
value1,value2,value3
'''
message = "Please generate the csv file of all tables. You can include some rows with empty values. You can separate the tables by table_n.csv: then the table in csv. Print only the csv files. For example "+example +"Keep the name of original headers"

# Call the predict function with the image and message, capturing the streaming output
full_response = ""
for response in predict(message, image):
  print(response, end="", flush=True)  # Print each part of the response as it's generated
  full_response += response

In [ ]:
full_response

In [ ]:
import csv

files_list = []

def extract_and_save_tables(full_response):
    """Extracts CSV tables from the full_response string and saves them as separate files."""
    current_table_name = None
    current_table_rows = []

    for line in full_response.splitlines():
        if line.startswith("Table "):
            if current_table_name:
                # Save the previous table
                save_table_to_csv(current_table_name, current_table_rows)
                files_list.append(current_table_name)  # Add file name to the list

            # Extract the table number to create the filename
            current_table_name = "table_" + line.split("Table ")[1].replace(":", "").strip() + ".csv"
            current_table_rows = []
        elif current_table_name:
            # If it's not an empty line, add it to the current table rows
            if line.strip():
                current_table_rows.append(line)

    # Save the last table
    if current_table_name:
        save_table_to_csv(current_table_name, current_table_rows)
        files_list.append(current_table_name)  # Add file name to the list

def save_table_to_csv(table_name, table_rows):
    """Saves a table to a CSV file."""
    try:
        with open(table_name, 'w', newline='', encoding='utf-8') as csvfile:
            writer = csv.writer(csvfile)

            # Write each row to the CSV file
            for row in table_rows:
                writer.writerow(row.split(","))
        print(f"Table saved as: {table_name}")
    except Exception as e:
        print(f"Error saving table {table_name}: {e}")
# Run the extraction and saving process
extract_and_save_tables(full_response)

# Print the list of created CSV files
print("Files created:", files_list)


In [ ]:
def display_first_5_rows(filename):
  """Displays the first 5 rows of a CSV file."""
  try:
    with open(filename, 'r', encoding='utf-8') as csvfile:
      reader = csv.reader(csvfile)
      rows = list(reader)
      for i in range(min(5, len(rows))):
        print(rows[i])
      print("-" * 20)  # Separator between files
  except FileNotFoundError:
    print(f"File not found: {filename}")
  except Exception as e:
    print(f"Error reading file {filename}: {e}")

# Loop through the files in the files_list and display the first 5 rows of each
for filename in files_list:
  print(f"File: {filename}")
  display_first_5_rows(filename)


# Front End Construction

In [ ]:
import gradio as gr
import os
import torch
from transformers import AutoProcessor, MllamaForConditionalGeneration, TextStreamer
from PIL import Image
import csv
# ... (Your existing code for model loading, predict function, etc.) ...
def process_image(image):
    example='''Table 1:
value1,value2,value3

Table 2:
value1,value2,value3
'''
    message = "Please generate the csv file of all tables. You can include some rows with empty values. You can separate the tables by table_n.csv: then the table in csv. Print only the csv files. For example "+example +"Keep the name of original headers"


    full_response = ""
    for response in predict(message, image):
        print(response, end="", flush=True)
        full_response += response

    files_list = []
    extract_and_save_tables(full_response)

    header_info = ""
    for filename in files_list:
        try:
            with open(filename, 'r', encoding='utf-8') as csvfile:
                reader = csv.reader(csvfile)
                rows = list(reader)
                if rows:
                    header_info += f"**{filename}:**\n"
                    header_info += ", ".join(rows[0]) + "\n\n"
        except FileNotFoundError:
            header_info += f"File not found: {filename}\n"
        except Exception as e:
            header_info += f"Error reading file {filename}: {e}\n"

    return header_info, files_list

def download_files(files_list):
    file_paths = [os.path.abspath(file) for file in files_list]
    return gr.File.update(value=file_paths, visible=True)

with gr.Blocks() as demo:
    with gr.Row():
        image_input = gr.Image(type="pil", label="Upload Image")
        with gr.Column():
            header_output = gr.Textbox(label="Headers of Extracted Tables")
            download_button = gr.File(label="Download CSV Files", visible=False)
    process_button = gr.Button("Extract Tables")
    process_button.click(fn=process_image, inputs=image_input, outputs=[header_output, download_button])
    download_button.change(fn=download_files, inputs=download_button, outputs=download_button)

demo.launch(debug=True)


In [ ]:
import gradio as gr
import os
import torch
from transformers import AutoProcessor, MllamaForConditionalGeneration, TextStreamer
from PIL import Image
import csv

# Check if we're running in a Hugging Face Space and if SPACES_ZERO_GPU is enabled
IS_SPACES_ZERO = os.environ.get("SPACES_ZERO_GPU", "0") == "1"
IS_SPACE = os.environ.get("SPACE_ID", None) is not None
IS_GDRIVE = True

# Determine the device (GPU if available, else CPU)
device = "cuda" if torch.cuda.is_available() else "cpu"
LOW_MEMORY = os.getenv("LOW_MEMORY", "0") == "1"
print(f"Using device: {device}")
print(f"Low memory mode: {LOW_MEMORY}")

# Get Hugging Face token from environment variables
HF_TOKEN = os.environ.get('HF_TOKEN')

# Define the model name
model_name = "Llama-3.2-11B-Vision-Instruct"
if IS_GDRIVE:
    model_path = "/content/drive/MyDrive/models/" + model_name
    model = MllamaForConditionalGeneration.from_pretrained(
        model_path,
        torch_dtype=torch.bfloat16,
        device_map="auto",
    )
    processor = AutoProcessor.from_pretrained(model_path)
else:
    model_name = "ruslanmv/" + model_name
    model = MllamaForConditionalGeneration.from_pretrained(
        model_name,
        use_auth_token=HF_TOKEN,
        torch_dtype=torch.bfloat16,
        device_map="auto",
    )
    processor = AutoProcessor.from_pretrained(model_name, use_auth_token=HF_TOKEN)

# Tie the model weights to ensure the model is properly loaded
if hasattr(model, "tie_weights"):
    model.tie_weights()

# Stream LLM response generator with minimal parameters
def stream_response(inputs, max_new_tokens=500, temperature=0.6):
    try:
        streamer = TextStreamer(tokenizer=processor.tokenizer)
        # Simplify the generation call with minimal parameters
        generated_tokens = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,  # Only keeping temperature
            do_sample=True,
            streamer=streamer  # Stream responses
        )
        for token in generated_tokens:
            yield processor.decode(token, skip_special_tokens=True)
    except Exception as e:
        print(f"Error during token generation: {e}")
        yield "Error during generation."

def predict(message, image, max_new_tokens=2000, temperature=1.0):
    try:
        messages = [
            {"role": "user", "content": [
                {"type": "image"},  # Specify that an image is provided
                {"type": "text", "text": message}  # Add the user-provided text input
            ]}
        ]

        input_text = processor.apply_chat_template(messages, add_generation_prompt=True)
        inputs = processor(image, input_text, return_tensors="pt").to(device)

        return stream_response(inputs, max_new_tokens=max_new_tokens, temperature=temperature)
    except Exception as e:
        print(f"Error in predict function: {e}")
        return ["Error in prediction."]

# Add function for extracting and saving tables
files_list = []

def extract_and_save_tables(full_response):
    current_table_name = None
    current_table_rows = []
    for line in full_response.splitlines():
        if line.startswith("Table "):
            if current_table_name:
                save_table_to_csv(current_table_name, current_table_rows)
                files_list.append(current_table_name)

            current_table_name = "table_" + line.split("Table ")[1].replace(":", "").strip() + ".csv"
            current_table_rows = []
        elif current_table_name:
            if line.strip():
                current_table_rows.append(line)

    if current_table_name:
        save_table_to_csv(current_table_name, current_table_rows)
        files_list.append(current_table_name)

def save_table_to_csv(table_name, table_rows):
    try:
        with open(table_name, 'w', newline='', encoding='utf-8') as csvfile:
            writer = csv.writer(csvfile)
            for row in table_rows:
                writer.writerow(row.split(","))
        print(f"Table saved as: {table_name}")
    except Exception as e:
        print(f"Error saving table {table_name}: {e}")

def display_first_5_rows(filename):
    try:
        with open(filename, 'r', encoding='utf-8') as csvfile:
            reader = csv.reader(csvfile)
            rows = list(reader)
            for i in range(min(5, len(rows))):
                print(rows[i])
            print("-" * 20)
    except FileNotFoundError:
        print(f"File not found: {filename}")
    except Exception as e:
        print(f"Error reading file {filename}: {e}")

# Setting environment variable for debugging CUDA
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
torch.backends.cudnn.benchmark = False


In [ ]:
import csv
import os
import gradio as gr
files_list = []
def extract_and_save_tables(full_response):
    """Extracts CSV tables from the full_response string and saves them as separate files."""
    current_table_name = None
    current_table_rows = []

    for line in full_response.splitlines():
        if line.startswith("Table "):
            if current_table_name:
                save_table_to_csv(current_table_name, current_table_rows)
                files_list.append(current_table_name)

            # Extract the table number to create the filename
            current_table_name = "table_" + line.split("Table ")[1].replace(":", "").strip() + ".csv"
            current_table_rows = []
        elif current_table_name:
            if line.strip():
                current_table_rows.append(line)

    if current_table_name:
        save_table_to_csv(current_table_name, current_table_rows)
        files_list.append(current_table_name)

def save_table_to_csv(table_name, table_rows):
    """Saves a table to a CSV file."""
    try:
        with open(table_name, 'w', newline='', encoding='utf-8') as csvfile:
            writer = csv.writer(csvfile)
            for row in table_rows:
                writer.writerow(row.split(","))
        print(f"Table saved as: {table_name}")
    except Exception as e:
        print(f"Error saving table {table_name}: {e}")

def process_image(image):
    example = '''Table 1:
    header1,header2,header3
    value1,value2,value3

    Table 2:
    value1,value2,value3
    '''
    message = "Please extract the table and generate the csv. If there are more tables separate the tables using table_n.csv. Then print table in csv. Print only the csv files. Please use name of original headers of the table. For example: " + example
    full_response = ""

    # Simulate the response prediction from LLM
    # Replace this with the actual logic from your model
    for response in predict(message, image):  # `predict` is assumed to be a generator-like function
        print(response, end="", flush=True)
        full_response += response

    extract_and_save_tables(full_response)

    header_info = ""
    for filename in files_list:
        try:
            with open(filename, 'r', encoding='utf-8') as csvfile:
                reader = csv.reader(csvfile)
                rows = list(reader)
                if rows:
                    header_info += f"**{filename}:**\n"
                    header_info += ", ".join(rows[0]) + "\n\n"
        except FileNotFoundError:
            header_info += f"File not found: {filename}\n"
        except Exception as e:
            header_info += f"Error reading file {filename}: {e}\n"

    # Return header information and the list of file paths
    file_paths = [os.path.abspath(file) for file in files_list]
    return header_info, file_paths

with gr.Blocks() as demo:
    with gr.Row():
        image_input = gr.Image(type="pil", label="Upload Image")
        with gr.Column():
            header_output = gr.Textbox(label="Headers of Extracted Tables")
            download_button = gr.File(label="Download CSV Files", file_count="multiple")
    process_button = gr.Button("Extract Tables")
    process_button.click(fn=process_image, inputs=image_input, outputs=[header_output, download_button])

demo.launch(debug=True)
